# Bayesian Model Selection

Authors: Yuchen Zhou and Audrey Olivier - 12/10/2018 <br>
Last modified by Audrey Olivier on 12/13/2018

## Quick review of Bayesian model selection

The problem of model selection consists in determining which model(s) best explain the available data $D$, given a set of candidate models $m_{1:M}$. Each model $m_{j}$ is parameterized by a set of parameters $\theta_{m_{j}} \in \Theta_{m_{j}}$, to be estimated based on data. In the Bayesian framework, model selection is perfomed by computing the posterior probability of each model $m_{j}$ using Bayes' theorem:

$$P(m_{j} \vert D) = \frac{p(D \vert m_{j})P(m_{j})}{\sum_{j=1}^{M} P(D \vert m_{j})P(m_{j})}$$

where $P(m_{j})$ is the prior assigned to model $m_{j}$ and $P(D \vert m_{j})$ is the model evidence, also called marginal likelihood.

$$ p(D \vert m_{j}) = \int_{\Theta_{m_{j}}} p(D \vert m_{j}, \theta_{m_{j}}) p(\theta_{m_{j}} \vert m_{j}) d\theta_{m_{j}} $$

where $p(\theta_{m_{j}} \vert m_{j})$ is the prior assigned to the parameter vector of model $m_{j}$.

## Numerical example

In the following we present an example for which the posterior pdf of the parameters, evidences and model probabilities can be computed analytically. We drop the $m_{j}$ subscript when referring to model parameters for simplicity. Three models are considered (the domain $x$ is fixed and consists in 50 equally spaced points):
\begin{align*}
m_{linear}:& \quad y = \theta_{0} x + \epsilon \\
m_{quadratic}:& \quad y = \theta_{0} x + \theta_{1} x^2 + \epsilon \\
m_{cubic}:& \quad y = \theta_{0} x + \theta_{1} x^2+ \theta_{2} x^3 + \epsilon \\
\end{align*}

All three models can be written in a compact form as $y=X \theta + \epsilon$, where $X$ contains the necessary powers of $x$. For all three models, the prior is chosen to be Gaussian, $p(\theta) = N(\cdot, \theta_{prior}, \Sigma_{prior})  $, and so is the noise $\epsilon \sim N(\cdot; 0, \sigma_{n}^{2} I)$. Then the posterior of the parameters can be computed analytically as:

\begin{align*}
& p(\theta \vert D={x,y}) =  N(\cdot; \theta_{post}(D), \Sigma_{post}(D)) \\
& \theta_{post}(D) = \left( \frac{1}{\sigma_{n}^{2}}X^{T}X + \Sigma_{prior}^{-1} \right)^{-1} \left(\frac{1}{\sigma_{n}^{2}}X^{T}y+\Sigma^{-1}\theta_{prior} \right) \\
& \Sigma_{post}(D) = \left( \frac{1}{\sigma_{n}^{2}}X^{T}X + \Sigma_{prior}^{-1} \right)^{-1}
\end{align*}

Then the evidence of each model can be computed as 

$$ p(D) = \frac{p(D \vert \theta)p(\theta)}{p(\theta \vert D)} $$
where $p(D \vert \theta) = N(\cdot; X\theta, \sigma_{n}^{2} I)$, $p(\theta) = N(\cdot, \theta_{prior}, \Sigma_{prior}) $ and $p(\theta \vert D) = N(\cdot, \theta_{post}(D), \Sigma_{post}(D))$. This formula can be computed at any point $\theta$.

### Generate data from the quadratic model

In [1]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Inference import *
from UQpy.RunModel import RunModel # required to run the quadratic model
from sklearn.neighbors import KernelDensity # for the plots
from statsmodels.nonparametric.kde import KDEUnivariate

In [2]:
# Generate data from a quadratic function
import random
random.seed(8) #set seed for reproducibility
from scipy.stats import multivariate_normal
param_true = np.array([1.0, 2.0]).reshape(1, -1)
var_n = 1
error_covariance = var_n*np.eye(50)
print(param_true.shape)

z = RunModel(samples=param_true, model_script='pfn_models.py', model_object_name = 'model_quadratic', 
             var_names = ['theta_1', 'theta_2'])
data = z.qoi_list[0].reshape((-1,))+multivariate_normal.rvs(mean=None, cov=error_covariance, size=1)
print(data)

(1, 2)
[  1.68443956e-01   3.23377297e-01   1.61412723e+00   3.09578314e+00
   2.33435386e+00   3.80180868e+00   4.15161476e+00   5.98375606e+00
   7.80701456e+00   7.26930083e+00   1.11722929e+01   1.21445941e+01
   1.43736715e+01   1.61677918e+01   2.01592462e+01   2.15401036e+01
   2.46469275e+01   2.76768394e+01   3.17499863e+01   3.56083132e+01
   3.69968372e+01   4.21928700e+01   4.38248416e+01   4.88103971e+01
   5.12975930e+01   5.64823141e+01   6.08208445e+01   6.61695470e+01
   6.97199585e+01   7.68601003e+01   8.20631947e+01   8.43504068e+01
   9.06800293e+01   9.69681202e+01   1.04522875e+02   1.07612268e+02
   1.14434110e+02   1.21318370e+02   1.27484935e+02   1.35638621e+02
   1.41720183e+02   1.47721792e+02   1.54746998e+02   1.63080678e+02
   1.69199636e+02   1.76060250e+02   1.86773302e+02   1.92493799e+02
   2.02235856e+02   2.10199971e+02]


### Define the models, compute the true values of the evidence.

For all three models, a Gaussian prior is chosen for the parameters, with mean and covariance matrix of the appropriate dimensions. Each model is given prior probability $P(m_{j}) = 1/3$.

In [3]:
# Define the models
model_names = ['model_linear', 'model_quadratic', 'model_cubic']
model_n_params = [1, 2, 3]
model_prior_means = [[0], [0,0], [0,0,0]]
model_prior_stds = [[10], [1,1], [1,2,0.25]]

In [4]:
evidences = []
model_posterior_means = []
model_posterior_stds = []
for n, model in enumerate(model_names):
    # compute matrix X
    X = np.linspace(0, 10, 50).reshape((-1,1))
    if n == 1: # quadratic model
        X = np.concatenate([X, X**2], axis=1)
    if n == 2: # cubic model
        X = np.concatenate([X, X**2, X**3], axis=1)

    # compute posterior pdf
    m_prior = np.array(model_prior_means[n]).reshape((-1,1))
    S_prior = np.diag(np.array(model_prior_stds[n])**2)
    S_posterior = np.linalg.inv(1/var_n*np.matmul(X.T,X)+np.linalg.inv(S_prior))
    m_posterior = np.matmul(S_posterior, 
                            1/var_n*np.matmul(X.T, data.reshape((-1,1)))+np.matmul(np.linalg.inv(S_prior),m_prior))
    m_prior = m_prior.reshape((-1,))
    m_posterior = m_posterior.reshape((-1,))
    model_posterior_means.append(list(m_posterior))
    model_posterior_stds.append(list(np.sqrt(np.diag(S_posterior))))
    print('posterior mean and covariance for '+model)
    print(m_posterior, S_posterior)
    
    # compute evidence, evaluate the formula at the posterior mean
    like_theta = multivariate_normal.pdf(data, mean=np.matmul(X,m_posterior).reshape((-1,)), cov=error_covariance)
    prior_theta = multivariate_normal.pdf(m_posterior, mean=m_prior, cov=S_prior)
    posterior_theta = multivariate_normal.pdf(m_posterior, mean=m_posterior, cov=S_posterior)
    evidence = like_theta*prior_theta/posterior_theta
    evidences.append(evidence)
    print('evidence for '+model+'= {}\n'.format(evidence))
    
# compute the posterior probability of each model
tmp = [1/3*evidence for evidence in evidences]
model_posterior_probas = [p/sum(tmp) for p in tmp]

print('posterior probabilities of all three models')
print(model_posterior_probas)

posterior mean and covariance for model_linear
[ 16.11858731] [[ 0.00059394]]
evidence for model_linear= 0.0

posterior mean and covariance for model_quadratic
[ 0.99382423  1.99640345] [[ 0.00941698 -0.00116538]
 [-0.00116538  0.00015392]]
evidence for model_quadratic= 3.9052574240850886e-34

posterior mean and covariance for model_cubic
[ 1.29285743  1.89676101  0.0074434 ] [[  5.60383740e-02  -1.66457308e-02   1.15371992e-03]
 [ -1.66457308e-02   5.29411877e-03  -3.83090725e-04]
 [  1.15371992e-03  -3.83090725e-04   2.85512405e-05]]
evidence for model_cubic= 4.909879048884739e-35

posterior probabilities of all three models
[0.0, 0.88831653646519537, 0.11168346353480459]


### Define the models for use in UQpy

In [5]:
# Define models
candidate_models = []
for n, model_name in enumerate(model_names):
    model = Model(n_params=model_n_params[n], model_type='python', 
                  model_script='pfn_models.py', model_object_name = model_name,
                  prior_name = ['normal']*model_n_params[n], 
                  prior_params = [[m, std] for (m,std) in zip(model_prior_means[n], model_prior_stds[n])],
                  error_covariance=error_covariance,
                  model_name=model_name)
    candidate_models.append(model)

### Run MCMC for all three models

In [6]:
# Linear model
from scipy.stats import norm

bayesMCMC = BayesParameterEstimation(data=data, model=candidate_models[0], sampling_method = 'MCMC', nsamples=1200,
                                     algorithm = 'MH', jump=10, nburn=100, pdf_proposal = 'Normal',
                                     pdf_proposal_scale = [0.1], 
                                     seed = model_prior_means[0]
                                     )
# plot prior, true posterior and estimated posterior
fig, ax = plt.subplots(figsize=(8,5))
domain_plot = np.linspace(0,20,200)
ax.plot(domain_plot, norm.pdf(domain_plot, loc=model_prior_means[0], scale=model_prior_stds[0]),
            label = 'prior', color='green', linestyle='--')
ax.plot(domain_plot, norm.pdf(domain_plot, loc=model_posterior_means[0], 
                                  scale=model_posterior_stds[0]),
            label = 'true posterior', color='red', linestyle='-')
ax.hist(bayesMCMC.samples[:,0], normed=True, bins=100, label='estimated posterior MCMC')
plt.legend()
plt.title('MCMC for linear model')
plt.show(fig)

AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
# Quadratic system
bayesMCMC = BayesParameterEstimation(data=data, model=candidate_models[1], sampling_method = 'MCMC', nsamples=3500,
                                     algorithm = 'MH', jump=10, nburn=100, pdf_proposal = 'Normal',
                                     pdf_proposal_scale = [0.1, 0.1], 
                                     seed = model_prior_means[1]
                                     )
# plot prior, true posterior and estimated posterior
fig, ax = plt.subplots(1,2,figsize=(16,5))
for n_p in range(2):
    domain_plot = np.linspace(-0.5,3,200)
    ax[n_p].plot(domain_plot, norm.pdf(domain_plot, loc=model_prior_means[1][n_p], scale=model_prior_stds[1][n_p]),
                label = 'prior', color='green', linestyle='--')
    ax[n_p].plot(domain_plot, norm.pdf(domain_plot, loc=model_posterior_means[1][n_p], 
                                      scale=model_posterior_stds[1][n_p]),
                label = 'true posterior', color='red', linestyle='-')
    ax[n_p].hist(bayesMCMC.samples[:,n_p], density=True, bins=30, label='estimated posterior MCMC')
    ax[n_p].legend()
    ax[n_p].set_title('MCMC for quadratic model')
plt.show(fig)

In [ ]:
# Cubic system
domain_plot = [np.linspace(-0.5,3,200), np.linspace(0,3,200), np.linspace(-0.5,0.5,200)]

bayesMCMC = BayesParameterEstimation(data=data, model=candidate_models[2], sampling_method = 'MCMC', nsamples=12000,
                                     algorithm = 'MH', jump=30, nburn=500, pdf_proposal = 'Normal',
                                     pdf_proposal_scale = [0.15, 0.1, 0.05], 
                                     seed = model_prior_means[2]
                                     )
# plot prior, true posterior and estimated posterior
fig, ax = plt.subplots(1,3,figsize=(24,5))
for n_p in range(3):
    ax[n_p].plot(domain_plot[n_p], norm.pdf(domain_plot[n_p], loc=model_prior_means[2][n_p], 
                                            scale=model_prior_stds[2][n_p]),
                label = 'prior', color='green', linestyle='--')
    ax[n_p].plot(domain_plot[n_p], norm.pdf(domain_plot[n_p], loc=model_posterior_means[2][n_p], 
                                      scale=model_posterior_stds[2][n_p]),
                label = 'true posterior', color='red', linestyle='-')
    ax[n_p].hist(bayesMCMC.samples[:,n_p], density=True, bins=30, label='estimated posterior MCMC')
    ax[n_p].legend()
    ax[n_p].set_title('MCMC for cubic model')
plt.show(fig)

### Run Bayesian Model Selection for all three models

In [ ]:
# Defines constants for the MCMC learning part, same as above
algos = ['MH']*3
proposal_types=['Normal']*3
scales = [[0.1],[0.1, 0.1],[0.15, 0.1, 0.05]]
nsamples = [1200,3500,12000]
nburn = [100,100,500]
jump = [10,10,30]

In [ ]:
selection = BayesModelSelection(data=data, candidate_models=candidate_models, pdf_proposal_type=proposal_types, 
                                pdf_proposal_scale=scales, algorithm=algos, jump=jump, nsamples=nsamples, nburn=nburn,  
                                prior_probabilities=[1./3., 1./3., 1./3.],
                                seed=model_prior_means, verbose=True)

In [ ]:
print('Sorted models:')
print(selection.sorted_model_names)
print('Evidence of sorted models:')
print(selection.sorted_evidences)
print('Posterior probabilities of sorted models:')
print(selection.sorted_probabilities)

As of version 2, the implementation of BayesModelSelection in UQpy uses the method of the harmonic mean to compute the models' evidence. This method is known to behave quite poorly, in particular it yeidls estimates with large variance. In the problem above, this implementation consistently detects that the quadratic model has the highest model probability, followed by the cubic model. However, the values of the evidence, and thus the model probabilities, are quite off and should not be trusted. Future versions of UQpy will integrate more advanced methods for the estimation of the evidence.